# Synthetic freight calculator 


## 1. Core config and data structures

In [1]:
from dataclasses import dataclass
from typing import Optional
import math

@dataclass
class Vessel:
    name: str
    dwt: float                 # deadweight in tonnes
    fuel_consumption_mt_per_day: float  # at service speed, laden
    ballast_factor: float      # % of laden consumption when in ballast
    speed_knots: float         # service speed
    daily_opex_usd: float      # crew, insurance, maintenance, etc.

@dataclass
class Route:
    load_port: str
    discharge_port: str
    laden_distance_nm: float   # nautical miles
    ballast_distance_nm: float # nautical miles
    port_costs_usd: float      # total ports (both ends)
    canal_costs_usd: float = 0.0

## Vessel

In [2]:
VLCC = Vessel(
    name="Generic VLCC",
    dwt=300_000,
    fuel_consumption_mt_per_day=70,  # laden
    ballast_factor=0.7,
    speed_knots=13.5,
    daily_opex_usd=12_000
)

LNGC_MEGI = Vessel(
    name="Generic LNGC MEGI",
    dwt=97_000,
    fuel_consumption_mt_per_day=110,  # including reliquefaction etc.
    ballast_factor=0.75,
    speed_knots=19.0,
    daily_opex_usd=35_000
)

## 2. Distance and bunker price placeholders

In [3]:
def get_distance_nm(load_port: str, discharge_port: str) -> float:
    """
    Placeholder for a distance API (e.g., ports.com).
    For now, hardcode or use a lookup table.
    """
    # Example crude: AG -> China VLCC route
    if (load_port, discharge_port) == ("Ras Tanura", "Ningbo"):
        return 6_300
    # Example LNG: Qatar -> Japan
    if (load_port, discharge_port) == ("Ras Laffan", "Tokyo"):
        return 6_800
    raise ValueError(f"No distance configured for {load_port} -> {discharge_port}")

def get_bunker_price_usd_per_mt(region: str, date: Optional[str] = None) -> float:
    """
    Placeholder for bunker price source (EIA, scraped, etc.).
    You can make this time-dependent later.
    """
    if region == "Singapore_VLSFO":
        return 650.0
    if region == "Fujairah_HSFO":
        return 500.0
    return 600.0

## 3. Voyage time and fuel consumption

In [4]:
def sailing_days(distance_nm: float, speed_knots: float) -> float:
    """
    Days = distance (nm) / (speed (knots) * 24)
    """
    return distance_nm / (speed_knots * 24.0)

def voyage_profile(vessel: Vessel, route: Route):
    """
    Compute days and fuel consumption for laden + ballast legs.
    """
    laden_days = sailing_days(route.laden_distance_nm, vessel.speed_knots)
    ballast_days = sailing_days(route.ballast_distance_nm, vessel.speed_knots)

    laden_fuel = laden_days * vessel.fuel_consumption_mt_per_day
    ballast_fuel = ballast_days * vessel.fuel_consumption_mt_per_day * vessel.ballast_factor

    total_days = laden_days + ballast_days
    total_fuel = laden_fuel + ballast_fuel

    return {
        "laden_days": laden_days,
        "ballast_days": ballast_days,
        "total_days": total_days,
        "laden_fuel_mt": laden_fuel,
        "ballast_fuel_mt": ballast_fuel,
        "total_fuel_mt": total_fuel
    }

## 4. Cost model and synthetic freight

In [5]:
def compute_voyage_cost(
    vessel: Vessel,
    route: Route,
    bunker_price_usd_per_mt: float
):
    profile = voyage_profile(vessel, route)

    fuel_cost = profile["total_fuel_mt"] * bunker_price_usd_per_mt
    opex_cost = profile["total_days"] * vessel.daily_opex_usd
    fixed_costs = route.port_costs_usd + route.canal_costs_usd

    total_cost = fuel_cost + opex_cost + fixed_costs

    return {
        "fuel_cost_usd": fuel_cost,
        "opex_cost_usd": opex_cost,
        "fixed_costs_usd": fixed_costs,
        "total_cost_usd": total_cost,
        "total_days": profile["total_days"],
        "profile": profile
    }

def freight_per_tonne_usd(
    total_cost_usd: float,
    cargo_tonnes: float
) -> float:
    return total_cost_usd / cargo_tonnes

def tce_usd_per_day(
    total_cost_usd: float,
    total_days: float,
    freight_revenue_usd: float
) -> float:
    """
    TCE = (Freight revenue - voyage costs) / voyage days
    Here we assume freight_revenue_usd is what the charterer pays.
    """
    return (freight_revenue_usd - total_cost_usd) / total_days

## 5. Putting it together: crude VLCC example

In [6]:
def example_vlcc_ag_china():
    # Build route with synthetic distances and port/canal costs
    laden = get_distance_nm("Ras Tanura", "Ningbo")
    ballast = laden  # assume round-trip symmetry for now

    route = Route(
        load_port="Ras Tanura",
        discharge_port="Ningbo",
        laden_distance_nm=laden,
        ballast_distance_nm=ballast,
        port_costs_usd=450_000,  # both ends, rough
        canal_costs_usd=0.0
    )

    bunker_price = get_bunker_price_usd_per_mt("Fujairah_HSFO")
    result = compute_voyage_cost(VLCC, route, bunker_price)

    # Assume full cargo ~ 2M barrels ~ 272k tonnes
    cargo_tonnes = 272_000
    freight_usd_per_t = freight_per_tonne_usd(result["total_cost_usd"], cargo_tonnes)

    print("=== VLCC AG -> China synthetic freight ===")
    print(f"Total days: {result['total_days']:.1f}")
    print(f"Total cost: {result['total_cost_usd']:,.0f} USD")
    print(f"Freight (cost) per tonne: {freight_usd_per_t:,.2f} USD/t")

    # If you want to back out an implied TCE from a market WS rate,
    # plug in a hypothetical freight revenue:
    hypothetical_freight_revenue = result["total_cost_usd"] * 1.25  # 25% margin
    tce = tce_usd_per_day(
        total_cost_usd=result["total_cost_usd"],
        total_days=result["total_days"],
        freight_revenue_usd=hypothetical_freight_revenue
    )
    print(f"Implied TCE: {tce:,.0f} USD/day")

## 6. LNG tanker example

In [7]:
def example_lng_qatar_japan():
    laden = get_distance_nm("Ras Laffan", "Tokyo")
    ballast = laden * 0.9  # assume slightly shorter ballast routing

    route = Route(
        load_port="Ras Laffan",
        discharge_port="Tokyo",
        laden_distance_nm=laden,
        ballast_distance_nm=ballast,
        port_costs_usd=600_000,
        canal_costs_usd=0.0
    )

    bunker_price = get_bunker_price_usd_per_mt("Singapore_VLSFO")
    result = compute_voyage_cost(LNGC_MEGI, route, bunker_price)

    # Assume ~170k m3 LNG, density ~0.45 t/m3 → ~76.5k tonnes
    cargo_tonnes = 76_500
    freight_usd_per_t = freight_per_tonne_usd(result["total_cost_usd"], cargo_tonnes)

    print("=== LNGC Qatar -> Japan synthetic freight ===")
    print(f"Total days: {result['total_days']:.1f}")
    print(f"Total cost: {result['total_cost_usd']:,.0f} USD")
    print(f"Freight (cost) per tonne: {freight_usd_per_t:,.2f} USD/t")